# 다중선형회귀 - 보스턴 집값 예측하기
## 다중선형회귀 수행
### 1) 패키지 준비

In [ ]:
import sys
sys.path.append('../../')
import helper

from pandas import read_excel, DataFrame, merge
from matplotlib import pyplot as plt
import seaborn as sb
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

### 2) 데이터셋 준비

|필드|설명|
|--:|-|
|CRIM|범죄율|
|ZN|25,000 평방피트를 초과 거주지역 비율|
|INDUS|비소매상업지역 면적 비율|
|CHAS|찰스강의 경계에 위치한 경우는 1, 아니면 0|
|NOX|일산화질소 농도|
|RM|주택당 방 수|
|AGE|1940년 이전에 건축된 주택의 비율|
|DIS|직업센터의 거리|
|RAD|방사형 고속도로까지의 거리|
|TAX|재산세율|
|PTRATIO|학생/교사 비율|
|B|인구 중 흑인 비율|
|LSTAT|인구 중 하위 계층 비율|
|MEDV|집값|
|CAT.MEDV|$3000 이상 여부|

In [ ]:
origin = read_excel('https://data.hossam.kr/E04/boston.xlsx')
origin.info()

### 3) 데이터 전처리
- 결측치 확인

In [2]:
origin.isna().sum()

NameError: name 'origin' is not defined

### 4) 탐색적 데이터 분석
- 명목형 처리

In [ ]:
origin['CHAS'] = origin['CHAS'].astype('category')
origin['CAT. MEDV'] = origin['CAT. MEDV'].astype('category')
origin.info()

- 기본통계 확인
    - `DataFrame.describe()`
    - 명목형 레이블들은 따로 파라미터에 넣지 않는 이상 default로 제외된다

In [3]:
origin.describe()

NameError: name 'origin' is not defined

- 상자그림

In [ ]:
plt.figure(figsize = (10, 5))
sb.boxplot(data=origin)
plt.show()
plt.close()

- 산점도 그래프와 추세선 확인

In [ ]:
plt.figure(figsize=(30, 30))
sb.pairplot(origin)
plt.show()
plt.close()

- 상관 계수 확인
    - 상관 계수: 두 변수 x, y 사이의 상관관계의 정도를 나타내는 수치
        - 상관계수 r은 항상 -1 ~ 1 사이에 있음
        - 상관계수 절대값의 크기는 직선 관계에 가까운 정도를 나타내고, 부호는 직선관계의 방향을 나타낸다
            - 상관계수의 절대값이 클수록(즉, 상관계수의 값이 1 또는 -1에 가까울수록) 두 변수 사이의 연관성이 크고, 0에 가까울수록 매우 약합을 의미
        - 종류
            - r > 0: 양의 상관관계
            - r < 0: 음의 상관관계
            - r = +1: 모든 점이 정확히 기울기가 양수인 직선 위에 위치한다
            - r = -1: 모든 점이 정확히 기울기가 음수인 직선 위에 위치한다
    - 주의할 점:
        - 자료분석시 큰 상관계수값이 항상 두 변수 사이의 어떤 인과관계를 의미하지는 않는다는 사실
        - 두 변수 사이에 연관성이 높다는 것이지, 뚜렷한 인과관계를 의미하지는 않는다는 의미
> 상관계수 내용 참조: https://leedakyeong.tistory.com/entry/%EA%B8%B0%EC%B4%88%ED%86%B5%EA%B3%84-%EC%83%81%EA%B4%80%EA%B3%84%EC%88%98%EB%9E%80-What-is-correlation-coefficient

In [ ]:
corr = origin.drop('MEDV', axis=1).corr()
corr

- 상관계수 히트맵

In [ ]:
plt.figure(figsize=(15, 10))
sb.heatmap(corr, 
           annot = True, \
           cmap = 'Greens', 
           vmin=-1, 
           vmax=1,
           linewidths=0.5)
plt.show()
plt.close()

- 제외하기로 한 독립변수 제거
> 상관계수의 절대값이 0.7 이상인 항목을 순차적으로 제거하여 회귀분석에 사용할 요인을 선정

In [4]:
x = origin.drop(['TAX', 'DIS', 'NOX', 'INDUS'], axis = 1)
x

NameError: name 'origin' is not defined

In [ ]:
plt.figure(figsize = (15, 10))
sb.heatmap(x.corr(), 
           annot=True, 
           cmap='Greens',
           vmin=-1,
           vmax=+1,
           linewidths=0.5)
plt.show()
plt.close()

In [ ]:
x.info()

- 명목형 변수만 추출

In [ ]:
categorical_df = x.filter(['CHAS', 'CAT. MEDV'])
categorical_df.info()

- 종속변수만 추출

In [ ]:
y = x['MEDV']
y

- 데이터프레임에서 추출한 필드를 제거

In [ ]:
x.drop(['CHAS', 'CAT. MEDV', 'MEDV'], axis = 1, inplace=True)
x.info()

- 최종 산점도 행렬 확인

In [ ]:
plt.figure(figsize = (30, 30))
sb.pairplot(x)
plt.show()
plt.savefig('pairplot.png')
plt.close()

### 5) 데이터셋 분할
- 랜덤시드 고정

In [ ]:
np.random.seed(777)

- 데이터 표준화

In [ ]:
x_scaler = StandardScaler()
x_scale = x_scaler.fit_transform(x)
x_scale

- 분리해 놓은 명목형 데이터와 결합
1. 명목형 변수의 값이 0, 1 두 종류 밖에 없기 때문에 별도로 더미변수 처리는 필요 없다
2. 표준화의 결과와 병합이 필요하지만 표준화 함수는 리턴타입이 numpy array이므로 이를 다시 DataFrame으로 변환해야만 명목형과 결합이 가능하다

In [ ]:
x_scale_df = DataFrame(x_scale, colulmns=x.columns)
x_scale_df

In [ ]:
x_df = merge(x_scale_df, categorical_df, left_index=True, right_index=True)
x_df.info()

- 훈련 데이터와 검증 데이터로 분할

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_df, 
                                                    y, 
                                                    test_size = 0.3,
                                                     random_state=777)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

### 6) 모델 개발
- 모델 정의

In [ ]:
my_model = Sequential()

# 1 차원의 데이터를 입력으로 받고, 64개의 출력을 가지는 첫 번째 Dense 층
my_model.add(Dense(64, activation='relu', input_shape=(len(x_train.columns),)))
my_model.add(Dense(32, activation='relu'))

# 하나의 값을 출력 정답의 범위가 정해지지 않기 때문에 활성화 함수는 linear
my_model.add(Dense(1, activation = 'linear'))

my_model.compilt(optimizer = 'adam', loss='mse', metrics=['mae'])
my_model.summary()

- 학습하기

In [ ]:
result = my_model.fit(x_train,
                      y_train, 
                      epochs=500, 
                      validation_data = (x_test, y_test),
                      callbacks = [EarlyStopping(monitor='val_loss',
                                                 patience=10,
                                                 verbose=1),
                                   ReduceLROnPlateau(monitor='val_loss',
                                                     patience=3,
                                                     factor = 0.5,
                                                     min_lr=0.001,
                                                     verbose = 1)])

### 7) 학습 결과 평가

In [ ]:
helper.tf_result_plot(result)

evaluate1 = my_model.evaluate(x_train, y_train)
print('최종 훈련 손실룰: %f, 최종 훈련 절대오차: %f' % (evaluate1[0], evaluate1[1]))

evaluate2 = my_model.evaluate(x_test, y_test)      
print('최종 검증 손실룰: %f, 최종 검증 절대오차: %f' % (evaluate2[0], evaluate2[1]))

### 8) 학습 결과 적용
- 테스트 데이터에 대한 예측 결과 산정

In [ ]:
train_pred = my_model.predict(x_train)
test_pred = my_model.predict(x_test)

- 결과 데이터셋 구성

In [ ]:
train_df = DataFrame(x_train, columns=x_train.columns)
train_df['관측치'] = y_train.values
train_df['예측치'] = train_pred.flatten()

train_df['훈련오차'] = train_df['관측치'] - train_df['예측치']

train_df.head()

In [ ]:
test_df = DataFrame(x_test, columns=x_test.columns)
test_df['관측치'] = y_test.values
test_df['예측치'] = test_pred.flatten()

test_df['훈련오차'] = test_df['관측치'] - test_df['예측치']

test_df.head()

- 실제 결과값과 머신러닝에 의한 예측값 비교

In [ ]:
for key in ['CRIM', 'ZN', 'RM', 'AGE', 'RAD', 'PTRATIO', 'B', 'LSTAT']:
    helper.regplot(train_df[key],
                   train_df['관측치'],
                   train_df['예측치'],
                   '훈련데이터',
                   test_df[key],
                   test_df['관측치'],
                   '검증데이터',
                   figsize = (15,5))